In [444]:
# read data as pandas df
# create new dictionary using user and timekey as keys
# give empty row to every user and timekey combination
# list time keys as 2017Q1 to 2020Q3
# sort by user and then sort by action date
# for each record
    # grab action date
    # calculate quarter of action date
    # walk backwards one quarter, add to dictionary and fill in all fields
    # add to dictionary and fill in all fields for current quarter
# convert dictionary to pandas
# fill forwards and fill backwards
# ensure no data exists past termination date
# take cross-sectional hr data
# select users who are not in the longitudinal data
# replicate record for every quarter
# add them into longitudinal data
# add gender race and date of birth in

import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
from datetime import date

data_dir = "/Users/Lara/Documents/Stanford/Research/Collabera/Data"
# this version contains race, age, and work country that Collabera_HR.csv does not contain
cs_hr_path = os.path.join(data_dir, "Collabera_HR.csv")
cs_perf_hr_path = os.path.join(data_dir, "Collabera_HR_Perf.csv")
user_qualtrics_path = os.path.join(data_dir, "qualtrics/UsersQualtrics.csv")
promotion_path = os.path.join(data_dir, "promotion_data.csv")
transfer_path = os.path.join(data_dir, "transfer_data.csv")
perf_percentage_path = os.path.join(data_dir, "perf_rating_percentages.csv")
perf_rating_path = os.path.join(data_dir, "perf_rating_likert.csv")

timekeys = [str(year)+"Q"+str(quarter) for year in range(2017, 2021) for quarter in range(1,5)]
timekeys.remove("2020Q4")


In [354]:
def date2quarter(date):
    """
    Return quarter of date in string
    """
    year, month = 0, 0
    date_obj = datetime.strptime(date, '%d-%b-%y')
    year = date_obj.year
    month = date_obj.month
    quarter = ((int(month)-1) // 3) + 1
    timekey = str(year) + 'Q' + str(quarter)
    return timekey

def calc_prev_quarter(quarter):
    """
    Return the quarter that immediately precedes current quarter
    """
    q = int(quarter[-1])
    if q > 1:
        prev_quarter = quarter[:-1]+str(q-1)
    else:
        year = int(quarter[0:4])
        prev_quarter = str(year-1)+'Q4'
    return prev_quarter

def calc_tenure(hire_date, quarter):
    """
    Calculate how many days have elapsed between hire date and the end of quarter
    """
    hire_obj = datetime.strptime(hire_date, '%d-%b-%y')
    curr_year = int(quarter[0:4])
    curr_month = 3*int(quarter[-1])
    curr_day = calendar.monthrange(curr_year, curr_month)[-1]
    curr_date_obj = datetime(curr_year, curr_month, curr_day)
    return (curr_date_obj - hire_obj).days

# Preparing Longitudinal Data Format

In [355]:
long_header = ['year', 'tenure', 'job_title', 'division', 'department',
          'function', 'work_location', 'work_state', 'work_country', 'legal_entity',
               'exit', 'exit_reason', 'promotion', 'salary_increase', 'demotion']
users = cs_df['UID'].to_list()
user_quarter2values = dict()
rows = list()
for u in users:
    for q in timekeys:
        user_quarter2values[(u, q)] = [int(q[0:4])] + [np.nan for i in range(9)] + [0, np.nan, 0, 0, 0]

# Processing Promotion Data

In [356]:
promotion_df = pd.read_csv(promotion_path)

promotion_df = promotion_df.drop(['PERSON ID', 'PERSON NUMBER', 'DOJ', "LINK"], axis=1)
names = promotion_df.columns.tolist()
promotion_df = promotion_df[['UID']+names[:-1]]
promotion_df['total_finite_fields'] = promotion_df.count(axis=1)
promotion_df['total_distinct_fields'] = promotion_df.apply(pd.Series.nunique, axis=1)

promotion_df = promotion_df.sort_values(by=['UID', 'EFFECTIVE DATE OF ACTION', 'total_finite_fields', 'total_distinct_fields'])
promotion_df = promotion_df.drop_duplicates(subset=['UID', 'EFFECTIVE DATE OF ACTION'], keep='last')
promotion_records = promotion_df.values.tolist()

This set of data has the following characteristics:
1) Sometimes there are multiple promotions in the same quarter.
2) Sometimes there are promotion events in consecutive quarters.
3) Sometimes there are multiple events on the same day for the same person. As this is not realistically plausible, this is a technical data error that needs to be corrected.

To circumvent these issues, the data is sorted by UID, date of action, number of finite fields, and number of distinct fields. Events that occurred on the same day are all dropped except for the event with the largest number of defined field, and the largest number of distinct fields if they all have the same number of defined fields.
Data from the latest promotion event is then used to label the job titles and units of a given quarter. Old titles and units are used to fill in the information from last quarter, IFF there is no pre-existing data there. This in practice means that if there are multiple events per quarter:
1) the last event (if multiple last events, the event with the largest # of finite and unique fields) determines titles of the current quarter)
2) the first event (f multiple first events, the event with the largest # of finite and unique fields), given that no event occurred last quarter that was used to fill in its data) determines the titles of the past quarter.


In [357]:
header = promotion_df.columns.tolist()
tenure_index = header.index('LENGTH OF SERVICE')
hire_index = header.index('GROUP DOJ')
action_index = header.index('ACTION CODE')
# effective date of action is the same as start date for every record looked at
quarter_index = header.index('EFFECTIVE DATE OF ACTION')
reason_index = header.index('ACTION REASON')
legal_index = header.index('NEW LEGAL ENTITY')
title_index = header.index('NEW JOB TITLE')
department_index = header.index('NEW DEPARTMENT')
division_index = header.index('NEW DIVISION')
function_index = header.index('NEW FUNCTION')
bu_index = header.index('NEW BUSINESS UNIT NAME')
city_index = header.index('NEW WORK CITY')
state_index = header.index("NEW WORK STATE")
country_index = header.index("NEW WORK COUNTRY")
termination_date_index = header.index("TERMINATION DATE")
termination_reason_index = header.index("TERMINATION REASON")

for record in promotion_records:
    uid = record[0]
    quarter = date2quarter(record[quarter_index])

    if (uid, quarter) in user_quarter2values:
        user_quarter2values[(uid, quarter)][1] = calc_tenure(record[hire_index], quarter)
        for i, index in enumerate([title_index, division_index, department_index, function_index,
                                   city_index, state_index, country_index, legal_index], 2):
            user_quarter2values[(uid, quarter)][i] = record[index]
        prev_quarter = calc_prev_quarter(quarter)
        
        # use action reason as ground truth if available (actions are coded as promotion when they are specified as demotions or when 
        # titles are specified as lateral)
        if type(record[reason_index]) == str:
            # demotion and salary increase are mutually exclusive
            if 'demotion' in record[reason_index].lower():
                user_quarter2values[(uid, quarter)][long_header.index('demotion')] = 1
            elif 'new salary' in record[reason_index].lower():
                user_quarter2values[(uid, quarter)][long_header.index('salary_increase')] = 1
            if 'promotion' in record[reason_index].lower():
                user_quarter2values[(uid, quarter)][long_header.index('promotion')] = 1
            # action codes seem to label someone as being promoted although action reason indicates that it is
            # a lateral title change with a new salary; we are trying to exclude this case
        elif 'promotion' in record[action_index].lower():
            user_quarter2values[(uid, quarter)][long_header.index('promotion')] = 1

    if (uid, prev_quarter) in user_quarter2values:
        if np.isfinite(user_quarter2values[(uid, prev_quarter)][1]):
            continue
        for i, index in enumerate([title_index, division_index, department_index, function_index,
                                   city_index, state_index, country_index, legal_index], 2):
                user_quarter2values[(uid, prev_quarter)][i] = record[index-1]
        user_quarter2values[(uid, prev_quarter)][1] = calc_tenure(record[hire_index], prev_quarter)
           
    if type(record[termination_date_index]) == str:
        termination_quarter = date2quarter(record[termination_date_index])
        if (uid, termination_quarter) in user_quarter2values:
            user_quarter2values[(uid, termination_quarter)][long_header.index('exit')] = 1
            user_quarter2values[(uid, termination_quarter)][long_header.index('exit_reason')] = record[termination_reason_index]
    


# Processing Transfer Data

In [358]:
transfer_df = pd.read_csv(transfer_path)
transfer_df = transfer_df.drop(['PERSON ID', 'PERSON NUMBER', 'DOJ', "LINK"], axis=1)
names = transfer_df.columns.tolist()
transfer_df = transfer_df[['UID']+names[:-1]]
transfer_df['total_finite_fields'] = transfer_df.count(axis=1)
transfer_df['total_distinct_fields'] = transfer_df.apply(pd.Series.nunique, axis=1)
transfer_df = transfer_df.sort_values(by=['UID', 'EFFECTIVE DATE OF ACTION', 'total_finite_fields', 'total_distinct_fields'])
transfer_df = transfer_df.drop_duplicates(subset=['UID', 'EFFECTIVE DATE OF ACTION'], keep='last')
transfer_records = transfer_df.values.tolist()
# all indices remain the same between promotion and transfer data


The transfer events are not as well formatted as the promotion events, and suffers more severely from the same issues. As a result, we prioritize records from the promotion data - if there are overlapping actions that take place in the same quarter, the data from the promotion data is used. Otherwise, it is processed in the same way as the promotion data.

To circumvent these issues, the data is sorted by UID, date of action, number of finite fields, and number of distinct fields. Events that occurred on the same day are all dropped except for the event with the largest number of defined field, and the largest number of distinct fields if they all have the same number of defined fields.
Data from the latest event is then used to label the job titles and units of a given quarter, IFF there is no data there (there might be data from promotion data). Old titles and units are used to fill in the information from last quarter, IFF there is no pre-existing data there.

In [359]:
for record in transfer_records:
    uid = record[0]
    quarter = date2quarter(record[quarter_index])
    if (uid, quarter) in user_quarter2values:
        # Avoiding overwriting any previously existing records
        if np.isfinite(user_quarter2values[(uid, quarter)][1]):
            continue
        user_quarter2values[(uid, quarter)][1] = calc_tenure(record[hire_index], quarter)
        for i, index in enumerate([title_index, division_index, department_index, function_index,
                                   city_index, state_index, country_index, legal_index], 2):
            user_quarter2values[(uid, quarter)][i] = record[index]

    prev_quarter = calc_prev_quarter(quarter)
    if (uid, prev_quarter) in user_quarter2values:
        if np.isfinite(user_quarter2values[(uid, prev_quarter)][1]):
            continue
        user_quarter2values[(uid, prev_quarter)][1] = calc_tenure(record[hire_index], prev_quarter)
        for i, index in enumerate([title_index, division_index, department_index, function_index,
                                   city_index, state_index, country_index, legal_index], 2):
            user_quarter2values[(uid, prev_quarter)][i] = record[index-1]
           
    if type(record[termination_date_index]) == str:
        termination_quarter = date2quarter(record[termination_date_index])
        if (uid, termination_quarter) in user_quarter2values:
            user_quarter2values[(uid, termination_quarter)][long_header.index('exit')] = 1
            user_quarter2values[(uid, termination_quarter)][long_header.index('exit_reason')] = record[termination_reason_index]

    # the action codes don't matter to us - all we care about is which unit the employee belonged to before and after

# Producing Longitudinal Dataframe

In [616]:
mux = pd.MultiIndex.from_tuples(user_quarter2values.keys())
long_hr = pd.DataFrame(list(user_quarter2values.values()), index=mux, columns=long_header)
cols = ['job_title', 'division', 'department', 'function', 'work_location', 'work_state', 'work_country', 'legal_entity']
long_hr.loc[:, cols] = long_hr.groupby(level=0)[cols].ffill()
long_hr.loc[:, cols] = long_hr.groupby(level=0)[cols].bfill()
long_hr = long_hr.sort_index()
long_hr.index.names = ['uid', 'quarter']
hire_dates = promotion_df.drop_duplicates(subset=['UID'])[['UID', 'GROUP DOJ']]
hire_dates = hire_dates.append(transfer_df.drop_duplicates(subset=['UID'])[['UID', 'GROUP DOJ']])
hire_dates = hire_dates.drop_duplicates()
hire_dates = hire_dates.set_index('UID')
hire_dates.index.name = 'uid'
long_hr = long_hr.join(hire_dates)

## Merging Cross-Sectional HR Data

In [617]:
cs_df = pd.read_csv(cs_hr_path)
cs_df.columns = ['uid', 'legal_entity', 'DOJ', 'GROUP DOJ', 'tenure', 'job_title', 'business_unit', 'division', 'department', 'function', 'work_location', 'work_state', 'year_of_birth', 'gender', 'yva', 'link']
cs_df = cs_df.drop(['yva', 'DOJ', 'link'], axis=1)
cs_perf_df = pd.read_csv(cs_perf_hr_path)
cs_perf_df.columns = ['uid', 'tenure', 'job_title', 'division', 'department', 'function', 'work_location', 'work_state', 'work_country', 'legal_entity', 'year_of_birth', 'gender', 'race', 'perf_2019', 'perf_2020', 'link']
cs_perf_df = cs_perf_df.drop(['perf_2019', 'perf_2020', 'link'], axis=1)

merged_cs_df = cs_df.merge(cs_perf_df, how='outer')
merged_cs_df.iloc[649] = merged_cs_df.iloc[649].fillna(merged_cs_df.iloc[1734])
# there is one duplicated row and the one that seems to be less useful is in row #1734, which we are dropping, after
# we paste its useful values to the original row
merged_cs_df = merged_cs_df.drop(1734)
merged_cs_df = merged_cs_df.set_index('uid')
merged_cs_df.index.name = 'uid'

In [618]:
long_hr = long_hr.combine_first(merged_cs_df)
long_hr['tenure'] = long_hr.apply(lambda row : calc_tenure(row['GROUP DOJ'], row.name[1]) if type(row['GROUP DOJ']) == str else np.nan, axis=1)

In [639]:
long_hr.loc[long_hr['work_country'] == 'US', 'work_country'] = 'U.S.A.'
long_hr.loc[long_hr['work_country'] == 'CA', 'work_country'] = 'Canada'
long_hr.loc[long_hr['work_country'] == 'IN', 'work_country'] = 'India'
long_hr.loc[long_hr['work_state'] == 'NC', 'work_state'] = 'North Carolina'
long_hr['work_location'] = long_hr['work_location'].str.replace('\(.+\)', '').str.strip()
long_hr.loc[long_hr['work_location'].str.contains('Bangalore'), 'work_location'] = 'Bangalore'
long_hr.loc[long_hr['race'].isnull(), 'race'] = 'Missing'
long_hr.loc[long_hr['race'].str.contains('Hispanic or Latino'), 'race'] = 'Hispanic or Latino'
long_hr.loc[long_hr['race'] == 'Race missing or unknown', 'race'] = 'Missing'

## Merging Performance Data

In [682]:
perf_likert_df = pd.read_csv(perf_rating_path)
perf_percentage_df = pd.read_csv(perf_percentage_path)
perf_likert_df = perf_likert_df[['UID', '2019 Perf_Type(Rating)', '2020 Perf_Type(Rating)']]
perf_percentage_df = perf_percentage_df[['UID', '2019_Perf_Type(Percentage)', '2020_Perf_Type(Percentage)']]
perf_likert_df.columns = ["uid", "perf_rating_2019", "perf_rating_2020"]
perf_percentage_df.columns = ["uid", "perf_percentage_2019", "perf_percentage_2020"]


In [683]:
perf_likert_df = pd.melt(perf_likert_df, id_vars='uid', value_vars=['perf_rating_2019', 'perf_rating_2020'])
perf_likert_df['year'] = perf_likert_df['variable'].apply(lambda var : 2019 if '2019' in var else 2020 if '2020' in var else np.nan)
perf_likert_df.drop('variable', inplace=True, axis=1)
perf_likert_df.columns = ['uid', 'perf_rating', 'year']

In [684]:
perf_percentage_df = pd.melt(perf_percentage_df, id_vars='uid', value_vars=['perf_percentage_2019', 'perf_percentage_2020'])
perf_percentage_df['year'] = perf_percentage_df['variable'].apply(lambda var : 2019 if '2019' in var else 2020 if '2020' in var else np.nan)
perf_percentage_df.drop('variable', inplace=True, axis=1)
perf_percentage_df.columns = ['uid', 'perf_percentage', 'year']

In [715]:
perf_df = perf_percentage_df.append(perf_likert_df)
perf_df = perf_df.set_index(['uid', 'year'])
perf_df.loc[perf_df['perf_rating'] == 'Not Applicable', 'perf_rating'] = np.nan
perf_df.loc[perf_df['perf_percentage'] == 'Not Applicable', 'perf_percentage'] = np.nan
perf_df['perf_percentage'] = perf_df['perf_percentage'].str.replace("%", "")
perf_df['perf_percentage'] = perf_df['perf_percentage'].apply(lambda val: float(val))

## Writing to File

In [731]:
long_hr.to_csv("/Users/Lara/Documents/CompCulture/spacespace/COco/analyses_data/longitudinal_hr.csv")

In [743]:
cs_df.head()

,uid,legal_entity,GROUP DOJ,tenure,job_title,business_unit,division,department,function,work_location,work_state,year_of_birth,gender
0,U2,Collabera Inc.,16-Apr-96,24.36,Chief Information Officer,Corporate-Default (0Z),Global Business Services,IT-Infrastructure,IT-Infrastructure,Basking Ridge (Allen Road),New Jersey,1965,Male
1,U4,Collabera Inc.,2-Dec-96,23.73,Senior Payroll Manager,Corporate-Default (0Z),Global Business Services,Finance,Payroll,Basking Ridge (Allen Road),New Jersey,1958,Female
2,U6,Collabera Inc.,12-Oct-98,21.87,Associate Vice President,Corporate-Default (0Z),Global Business Services,Immigration,Immigration,Basking Ridge (Allen Road),New Jersey,1967,Male
3,U7,Collabera Inc.,1-Feb-99,21.56,Executive Vice President,Ashwin-SAT Get (4A),Strategic Accounts Team,Sales,Sales,Basking Ridge (Allen Road),New Jersey,1970,Male
4,U8,Collabera Inc.,31-Mar-99,21.40,Delivery Manager,Regional-Tyler (9C),Regional,Recruitment,Recruitment,Basking Ridge (Allen Road),New Jersey,1971,Male
